In [46]:
from No_sync.Frost_api import client_id, client_secret
import requests
import pandas as pd
from datetime import datetime, timedelta

In [47]:
# SN65320, SN65310, SN65370 
# Closest station gave error, so I used the second closest station
# air_temperature, precipitation_amount, wind_speed, relative_humidity

In [48]:
# Define the endpoint and parameters
endpoint = 'https://frost.met.no/observations/v0.jsonld'

# Calculate the date range for the last year
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# Format the date range as a string
date_range = start_date.strftime('%Y-%m-%d') + '/' + end_date.strftime('%Y-%m-%d')

parameters = {
    'sources': 'SN65310',
    'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D),mean(relative_humidity P1D)',
    'referencetime': date_range,
}

# Issue an HTTP GET request
r = requests.get(endpoint, params=parameters, auth=(client_id, ''))

# Extract JSON data
json = r.json()


In [49]:
# Check if the request worked, print out any errors
if r.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json['error']['message'])
    print('Reason: %s' % json['error']['reason'])

Data retrieved from frost.met.no!


In [50]:
df_total = pd.DataFrame()
for i in range(len(data)):
    row = pd.DataFrame(data[i]['observations'])
    row['referenceTime'] = data[i]['referenceTime']
    row['sourceId'] = data[i]['sourceId']

    df_total = pd.concat([row, df_total])

In [ ]:
# These additional columns will be kept
columns = ['sourceId','referenceTime','elementId','value','unit','timeOffset']
df = df_total[columns].copy()
# Convert the time value to something Python understands
df['referenceTime'] = pd.to_datetime(df['referenceTime'])

In [45]:
df

,sourceId,referenceTime,elementId,value,unit,timeOffset
0,SN65310:0,2023-10-12 00:00:00+00:00,mean(air_temperature P1D),9.7,degC,PT0H
1,SN65310:0,2023-10-12 00:00:00+00:00,mean(air_temperature P1D),9.2,degC,PT6H
2,SN65310:0,2023-10-12 00:00:00+00:00,mean(wind_speed P1D),14.5,m/s,PT0H
3,SN65310:0,2023-10-12 00:00:00+00:00,mean(relative_humidity P1D),76.0,percent,PT0H
0,SN65310:0,2023-10-11 00:00:00+00:00,mean(air_temperature P1D),9.4,degC,PT0H
...,...,...,...,...,...,...
3,SN65310:0,2022-10-14 00:00:00+00:00,mean(relative_humidity P1D),80.0,percent,PT0H
0,SN65310:0,2022-10-13 00:00:00+00:00,mean(air_temperature P1D),9.3,degC,PT0H
1,SN65310:0,2022-10-13 00:00:00+00:00,mean(air_temperature P1D),10.1,degC,PT6H
2,SN65310:0,2022-10-13 00:00:00+00:00,mean(wind_speed P1D),6.2,m/s,PT0H
